<a href="https://colab.research.google.com/github/Nomi-lpr/skills-communicate-using-markdown/blob/main/happy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip mcp-tool-api-main.zip

Archive:  mcp-tool-api-main.zip
ba5d5d88f7bab6f7f6d749a14d9633636c3fef94
   creating: mcp-tool-api-main/
  inflating: mcp-tool-api-main/.gitignore  
   creating: mcp-tool-api-main/configs/
  inflating: mcp-tool-api-main/configs/both_tools_config.json  
  inflating: mcp-tool-api-main/configs/crop_only_config.json  
 extracting: mcp-tool-api-main/configs/no_tools_config.json  
  inflating: mcp-tool-api-main/configs/video_only_config.json  
  inflating: mcp-tool-api-main/data_postprocess.py  
  inflating: mcp-tool-api-main/main.py  
  inflating: mcp-tool-api-main/mcp_api_usage.py  
  inflating: mcp-tool-api-main/mcp_utils.py  
  inflating: mcp-tool-api-main/recheck_score.py  
  inflating: mcp-tool-api-main/tool_test.py  
   creating: mcp-tool-api-main/tools/
 extracting: mcp-tool-api-main/tools/__init__.py  
  inflating: mcp-tool-api-main/tools/crop_server.py  
  inflating: mcp-tool-api-main/tools/grounding_server.py  
  inflating: mcp-tool-api-main/tools/video_server.py  
   creating: mc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
! ls



drive  mcp-tool-api-main  mcp-tool-api-main.zip  sample_data


In [6]:
!pip install openai httpx numpy decord Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 90.6 MB/s eta 0:00:00


In [7]:
%cd /content/mcp-tool-api-main/

/content/mcp-tool-api-main


In [10]:
!pip install mcp==1.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00


In [11]:
import asyncio
import json
import os
from mcp_api_usage import MCPClient
from mcp_utils import format_conversation

In [12]:
VIDEO_DIR = "./my_videos/"